# DicoCo

*Thomas CAILLET*

In [1]:
# import necessaire à la création des widgets
from IPython.display import display, clear_output, HTML, Markdown, Audio
import ipywidgets as widgets

In [2]:
import pandas as pd
import re

In [3]:
from requests_html import HTMLSession
import re
import pandas as pd
from bs4 import BeautifulSoup

In [4]:
# Fonction de scrapping depuis longman reprise de la partie 1
def scrap_from_longman(word):
    
    # requête http
    URL = 'https://www.ldoceonline.com/dictionary/'+word
    session = HTMLSession()
    response = session.get(URL)
    
    if (response.status_code == 200):
        
        # récupération du body
        html_doc = response.content
        body = BeautifulSoup(html_doc,'html.parser').find('body')
        
        # instanciation des listes de données à récupérer
        list_type = []
        list_pron = []
        list_audio_br = []
        list_audio_us = []
        
        # récupération des spans contenant les données recherchées
        frq_head = body.findAll('span', {'class': re.compile('(frequent )*Head')})

        for result in frq_head:
            # recherche des spans des valeurs POS et PRON dans les sous-arbres
            if result.find('span', {'class':'POS'}) is not None:
                list_type.append(result.find('span', {'class':'POS'}).text.strip())
            if result.find('span', {'class':'PRON'}) is not None:
                list_pron.append(result.find('span', {'class':'PRON'}).text.strip())

            spans_of_freq_head = result.findAll('span')
            
            # recherche des spans ayant comme attribut data-src-mp3
            for span in spans_of_freq_head:
                if span.has_attr('data-src-mp3'):
                    if 'British' in span['title']:
                        list_audio_br.append(span['data-src-mp3'])
                    elif 'American' in span['title']:
                        list_audio_us.append(span['data-src-mp3'])
        
        # nombre d'homonymes
        nb_hom = len(list_type)
        
        # création du dataframe final
        list_mots = nb_hom * [word]
        while (len(list_pron) < nb_hom):
            list_pron.append(list_pron[0])
        
        df_result = pd.DataFrame(list(zip(list_mots, list_type, list_pron, list_audio_br, list_audio_us)),
                        columns = ['mot', 'type', 'pron', 'audio_br', 'audio_us'])
    else:
        # on retourne quand même un tableau vide concatenable
        df_result = pd.DataFrame(columns = ['mot', 'type', 'pron', 'audio_br', 'audio_us'])
    
    return df_result

In [5]:
cheminDicoBuchMot = "./../data/dataframe/BuchananRich.csv"
monDico = pd.read_csv(cheminDicoBuchMot, delimiter=";", encoding="utf8")[['hwd', 'pron', 'POS', 'pronSampa', 'ipa']]

In [6]:
def filtDf(strHwd):    
    dfMonDicoFiltered = monDico[monDico['hwd'].str.contains(strHwd, na=False,flags=re.IGNORECASE, regex=True)]
    return dfMonDicoFiltered

In [7]:
# widget input
textFilter = widgets.Text(
       value="love",
       description='filtre hwd',
       layout={'width': '300px'} )

# widget button
buttonFiltrer = widgets.Button(description='Filtrer')

# widget slider nb occurences
nbOccur_f = widgets.IntSlider(value=3, min=0, max=20)

# widgets d'outputs
outDf = widgets.Output()
outMp3 = widgets.Output()

# fonction appelé au click
def on_button_clicked_f(_):
    
    # récupérer le texte saisi par l'utilisateur
    targetWord = textFilter.value
    df_filtered = filtDf(targetWord)
    # on préfère ici afficher les n premiers plutôt qu'un sample
    df_filtered_visible = df_filtered.head(nbOccur_f.value)
    
    # widget affichant le Df
    with outDf:
        clear_output()
        display(df_filtered_visible)
    
    # widget contenant l'url longman et les players audios
    with outMp3:
        clear_output()
        for word in df_filtered_visible['hwd']:
            urlLongman = "https://www.ldoceonline.com/dictionary/"
            urlTarget = urlLongman+word
            display(HTML('<a href="'+urlTarget+'" target="_blank" >'+'longman : '+word+'</a>'))
            
            # appel de la fonction de scrapping renvoyant un Df (pouvant être nul!)
            audio_url_br = scrap_from_longman(word)['audio_br']
            audio_url_us = scrap_from_longman(word)['audio_us']
            
            # on split l'url qui contient la version en paramètre '?version=XXX' 
            # --> url .mp3 compatible avec le display Audio
            if len(audio_url_br) > 0:
                audio_url_br = audio_url_br.iloc[0].split('?', 1)[0]
                display(HTML('<b>BR</b>'))
                display(Audio(url=audio_url_br))
            
            if len(audio_url_us) > 0:
                audio_url_us = audio_url_us.iloc[0].split('?', 1)[0]
                display(HTML('<b>US</b>'))
                display(Audio(url=audio_url_us))
            
            if len(audio_url_br) == 0 and len(audio_url_us) == 0:
                display(HTML('Pas de fichier audio pour le mot '+word))
            
buttonFiltrer.on_click(on_button_clicked_f)

In [8]:
vbFiltres_f = widgets.HBox([textFilter, buttonFiltrer, nbOccur_f])
vbOuts_f = widgets.VBox([outDf,outMp3])

hbAllWidgets_f = widgets.VBox([vbFiltres_f,vbOuts_f])

hbAllWidgets_f